In [9]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [10]:
# 함수 정의

from datetime import datetime

def calculate_age(birthdate):
    
    birth = datetime.strptime(birthdate, "%Y-%m-%d")
    today = datetime.today()
    age = today.year - birth.year
    # 아직 생일이 안 지났으면 1살 빼기
    if (today.month, today.day) < (birth.month, birth.day):
        age -= 1
    return age

def convert_currency(amount, rate):
    return round(amount * rate, 2)

def calculate_bmi(height, weight):
    # cm를 m로 변환
    height_m = height / 100 
    bmi = weight / (height_m ** 2)
    return round(bmi, 2)

In [11]:
tools = [
    {
        "type": "function",
        "name": "calculate_age",
        "description": "YYYY-MM-DD 형식으로 만 나이를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "birthdate": {
                    "type": "string",
                    "description": "생년월일 (YYYY-MM-DD)"
                }
            },
            "required": ["birthdate"],
            "additionalProperties": False
        }
    },
    {
        "type": "function",
        "name": "convert_currency",
        "description": "금액과 환율로 환전 결과를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "변환할 금액"},
                "rate": {"type": "number", "description": "적용할 환율"}
            },
            "required": ["amount", "rate"],
            "additionalProperties": False
        }
    },
    {
        "type": "function",
        "name": "calculate_bmi",
        "description": "키(cm)와 몸무게(kg)로 BMI를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "number", "description": "키(cm)"},
                "weight": {"type": "number", "description": "몸무게(kg)"}
            },
            "required": ["height", "weight"],
            "additionalProperties": False
        }
    }
]

In [12]:
input_msgs = [
        {"role": "user", "content": "2010-10-03생의 만 나이는?"},
        {"role": "user", "content": "500 달러를 원화로 바꿔 줘 (환율 1330원 적용)"},
        {"role": "user", "content": "키 185cm, 몸무게 70kg의 BMI는?"}
    ]

response = client.responses.create(
    model="gpt-4.1",
    tools=tools,
    input=input_msgs
)

In [13]:
response.output

[ResponseFunctionToolCall(arguments='{"birthdate":"2010-10-03"}', call_id='call_xL8jTBX9izQ4saNIME8oaBpB', name='calculate_age', type='function_call', id='fc_685c9a035174819b8a818ae3ad4b02aa0646a6cd74c54e2d', status='completed'),
 ResponseFunctionToolCall(arguments='{"amount":500,"rate":1330}', call_id='call_86SKkj2zGaCjs3ckMSTtxLvy', name='convert_currency', type='function_call', id='fc_685c9a036b18819b8bd8ba09060214da0646a6cd74c54e2d', status='completed'),
 ResponseFunctionToolCall(arguments='{"height":185,"weight":70}', call_id='call_fY3LNgOjfA2QSi9sTG07Gzan', name='calculate_bmi', type='function_call', id='fc_685c9a038398819b84d02ee00d7fbb730646a6cd74c54e2d', status='completed')]

In [14]:
import json

tool_call1 = response.output[0]
args1 = json.loads(tool_call1.arguments)

tool_call2 = response.output[1]
args2 = json.loads(tool_call2.arguments)

tool_call3 = response.output[2]
args3 = json.loads(tool_call3.arguments)

tool_calls = response.output

result1 = calculate_age(args1['birthdate'])
result2 = convert_currency(
    args2['amount'],
    args2['rate']
)
result3 = calculate_bmi(args3['height'], args3['weight'])

In [15]:
function_outputs = [
    {
        'type': 'function_call_output',
        'call_id': str(tool_call1.call_id),
        'output': str(result1)
    },
    {
        'type': 'function_call_output',
        'call_id': str(tool_call2.call_id),
        'output': str(result2)
    },
    {
        'type': 'function_call_output',
        'call_id': str(tool_call3.call_id),
        'output': str(result3)
    }
]

final_input_msgs = input_msgs + list(tool_calls) + function_outputs

# 최종 응답 요청
response_2 = client.responses.create(
    model="gpt-4.1",
    input=final_input_msgs,
    tools=tools,
)

print(response_2.output_text)

요청하신 정보는 다음과 같습니다:

1. 2010-10-03생의 만 나이: 14세입니다.
2. 500달러를 원화(환율 1330원)로 환전한 금액: 665,000원입니다.
3. 키 185cm, 몸무게 70kg의 BMI: 20.45입니다. (정상 체중 범위입니다)
